<a href="https://colab.research.google.com/github/Pankaj-2003/Strategy_Performance_Python/blob/main/Quant_Analyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime , timedelta

## Dataframe columns
### Date (datetime64[ns]): The date and time of the trade
### Symbol (string): The ticker symbol of the traded security
### Side (string): Either 'buy' or 'sell'
### Size (float, optional): The number of shares traded (default to 1 if not provided)
### Price (float): The price at which the trade was executed

In [ ]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    # This function returns the price of the security 'ticker' at the given 'date'
    # For the purpose of this exercise, assume it returns a random number
    return random.uniform(1, 100)  # Example implementation

In [ ]:
metrics = ["Sharpe" , "max-drawdown" ,  "profit factor" , "total net profit" , "Percent Profitable" , "average win\loss Ratio" , "Expectancy" , "Recovery factor" , "average r multiple" , "ROI"]

In [ ]:
def calculate_sharpe(daily_returns, risk_free_rate=0.01):
    average_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()

    # Avoid division by zero
    if std_daily_return == 0:
        return np.inf

    sharpe_ratio = (average_daily_return - risk_free_rate) / std_daily_return
    return sharpe_ratio

In [ ]:
def calculate_max_drawdown(daily_returns):
    cumulative_return = (1 + daily_returns).cumprod()
    cumulative_max = cumulative_return.cummax()
    drawdown = cumulative_return - cumulative_max
    max_drawdown = drawdown.min()

    return max_drawdown

In [ ]:
def calculate_profit_factor(trades):
    winning_trades = trades[trades['Return'] > 0]
    losing_trades = trades[trades['Return'] < 0]

    total_profit = winning_trades['Return'].sum()
    total_loss = -losing_trades['Return'].sum()  # Negate to get positive loss value

    if total_loss == 0:
        return np.inf if total_profit > 0 else 0  # Handle edge cases

    profit_factor = total_profit / total_loss
    return profit_factor


In [ ]:
def percent_profitable(trades):
  winning_trades = trades[trades["Return"] > 0]
  total_trades = len(trades)
  percent_profitable = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0
  return percent_profitable

In [ ]:
def average_win_loss_ratio(trades):
    average_win = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
    average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0

    average_win_loss_ratio = average_win / average_loss if average_loss  != 0 else 0
    return average_win_loss_ratio


In [ ]:
def expectancy(trades):
  average_profit = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
  average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0
  winning_trades = trades[trades["Return"] > 0]
  losing_trades = trades[trades["Return"] < 0]
  total_trades = len(trades)
  win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
  loss_rate = len(losing_trades) / total_trades if total_trades > 0 else 0

  expectancy = (win_rate * average_profit) - (loss_rate * average_loss)
  return expectancy

In [ ]:
def recovery_factor(trades  ,daily_returns):
  total_profit = trades[trades['Return'] > 0]['Return'].sum()
  total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  total_net_profit = total_profit - total_loss
  max_dd = calculate_max_drawdown(daily_returns)
  recovery_factor = total_net_profit / abs(max_dd) if max_dd != 0 else np.inf
  return recovery_factor

In [ ]:
def trade_perf(trades : pd.DataFrame) -> pd.Series:
  # if dataframe is empty
  if trades.empty:
    return pd.Series({
            'Sharpe': 0,
            'max-drawdown': 0,
            'profit factor': 0,
            'total net profit': 0,
            'Percent Profitable': 0,
            'average win\loss Ratio': 0,
            'Expectancy': 0,
            'Recovery factor': 0,
            'volatility': 0,
            'ROI': 0})

  # handling missing values

  trades['Size'] = trades['Size'].fillna(1)
  trades = trades[trades['Symbol'].notna()]
  trades = trades[trades['Side'].notna()]

  # seperate way to fill price
  def fill_missing_prices(row):
    if pd.isna(row['Price']):
        row['Price'] = getTickerPrice(row['Symbol'], row['Date'])
    return row
  trades = trades.apply(fill_missing_prices, axis=1)




  # calculating returns
  def calculate_return(row):
      current_date = datetime.now().strftime('%Y-%m-%d')
      current_price = getTickerPrice(row['Symbol'], current_date)
      if row['Side'] == 'buy':
          return (current_price - row['Price']) * row['Size']
      else:
          return (row['Price'] - current_price) * row['Size']

  trades['Return'] = trades.apply(calculate_return, axis=1 )  # Applying the function to each row of the df
  daily_returns = trades.groupby('Date')['Return'].sum() # daily return for sharpe


  # 1- calculating metrics

  sharpe_ratio = calculate_sharpe(daily_returns)
  print(trades)

  # 2 - max_drawdown
  maxdd = calculate_max_drawdown(daily_returns)

  # 3 - profit factor
  profit_factor = calculate_profit_factor(trades)

  # 4 - total net profit
  total_profit = trades[trades['Return'] > 0]['Return'].sum()
  total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  total_net_profit = total_profit - total_loss

  # 5 - percent profitable
  perc_profitable = percent_profitable(trades)
  # 6 - average win/loss ratio
  avg_win_loss_ratio = average_win_loss_ratio(trades)


  # 7 - expectancy
  expectncy = expectancy(trades)

  # 8 - recovery factor
  recovery_factr = recovery_factor(trades  ,daily_returns)

  # 9 - volatility
  volatility = trades['Return'].std() #we can calculate stock wise volatility using  trades.groupby('Symbol')['Return'].std()


  # 10 - roi
  total_investement = (trades["Price"]* trades["Size"]).sum()
  roi = (total_net_profit / total_investement) * 100 if total_investement != 0 else 0


  metrics = pd.Series({
            'Sharpe': sharpe_ratio,
            'max-drawdown': maxdd,
            'profit factor': profit_factor,
            'total net profit': total_net_profit,
            'Percent Profitable': perc_profitable,
            'average win\loss Ratio': avg_win_loss_ratio,
            'Expectancy': expectncy,
            'Recovery factor': recovery_factr,
            'volatility': volatility,
            'ROI': roi})
  metrics_rounded = metrics.round(2)
  metrics = metrics_rounded.apply(lambda x: f"{x:.2f}")

  return metrics

In [ ]:
# symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
# sides = ['buy', 'sell']
# base_date = datetime(2024, 7, 15)
# dates = [base_date + timedelta(days=i//4) for i in range(20)]
# data = {
#     'Date': dates,
#     'Symbol': [random.choice(symbols) for _ in range(20)],
#     'Side': [random.choice(sides) for _ in range(20)],
#     'Size': [random.randint(1, 100) for _ in range(20)],
#     'Price': [random.uniform(50, 300) for _ in range(20)],
# }
# trades = pd.DataFrame(data)




In [ ]:
# # Introduce missing values randomly
# for col in ['Symbol', 'Size', 'Price']:
#     trades.loc[random.sample(range(len(trades)), k=5), col] = np.nan  # Set 5 random entries in each column to NaN


### How did i compute returns in case of long short strategies
## i first get the current price of stock using auxiliary function , then for buy side we do current price - (buy_price * size) and vice versa for sell side

In [ ]:
test = pd.read_csv('testData.csv')

In [ ]:
# preprocessing
test['transactionDate'] = test['transactionDate'].str.replace('-', '/')
test['transactionDate'] = pd.to_datetime(test['transactionDate'], format='%m/%d/%Y')
test['ticker'] = test['ticker'].astype(str)
test['type'] = test['type'].map({'Purchase': 'buy', 'Sale (Full)': 'sell', 'Sale (Partial)': 'sell'})

In [ ]:
test.dtypes

disclosureYear                     int64
disclosureDate                    object
transactionDate           datetime64[ns]
owner                             object
ticker                            object
assetDescription                  object
type                              object
amount                            object
representative                    object
district                          object
capitalGainsOver200USD            object
option_symbol                     object
dtype: object

In [ ]:
# extracting lower bound
test['amount'] = test['amount'].str.replace('$', '').str.replace(',', '')
test['amount'] = test['amount'].apply(lambda x: float(x.split(' - ')[0]) if isinstance(x, str) else x)

In [ ]:
test

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol
0,2023,6/15/2023,2023-05-20,Spouse,AAPL,Apple Inc. Stock,buy,100001.0,Nancy Pelosi,CA-12,Yes,NaN
1,2023,05-12-2023,2023-04-10,Self,GOOGL,Alphabet Inc. Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN
2,2023,12/29/2023,2022-12-06,Dependent,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,NY-14,Yes,NaN
3,2023,2/13/2022,2022-04-14,Self,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,FL-9,No,NaN
4,2023,6/23/2023,2022-03-11,Joint,MSFT,MSFT Stock,buy,1001.0,Nancy Pelosi,CA-12,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,2023,3/15/2023,2023-07-26,Dependent,AAPL,AAPL Stock,sell,1001.0,Nancy Pelosi,FL-9,Yes,NaN
98,2022,06-08-2022,2023-05-03,Joint,TSLA,TSLA Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN
99,2023,7/24/2022,2023-09-16,Self,GOOGL,GOOGL Stock,sell,1001.0,Nancy Pelosi,TX-7,No,NaN
100,2023,12/25/2022,2022-06-25,Spouse,AAPL,AAPL Stock,sell,100001.0,Nancy Pelosi,NY-14,Yes,NaN


In [ ]:
test['price'] = test.apply(lambda row: getTickerPrice(row['ticker'], row['transactionDate']), axis=1) # here we assume that nancy would have bought on the price of transaction day , which is not accurate but close.


,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol,price
0,2023,6/15/2023,2023-05-20,Spouse,AAPL,Apple Inc. Stock,buy,100001.0,Nancy Pelosi,CA-12,Yes,NaN,86.019421
1,2023,05-12-2023,2023-04-10,Self,GOOGL,Alphabet Inc. Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN,8.773509
2,2023,12/29/2023,2022-12-06,Dependent,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,NY-14,Yes,NaN,65.104681
3,2023,2/13/2022,2022-04-14,Self,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,FL-9,No,NaN,64.400713
4,2023,6/23/2023,2022-03-11,Joint,MSFT,MSFT Stock,buy,1001.0,Nancy Pelosi,CA-12,Yes,NaN,35.082036
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2023,3/15/2023,2023-07-26,Dependent,AAPL,AAPL Stock,sell,1001.0,Nancy Pelosi,FL-9,Yes,NaN,87.272859
98,2022,06-08-2022,2023-05-03,Joint,TSLA,TSLA Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN,33.837085
99,2023,7/24/2022,2023-09-16,Self,GOOGL,GOOGL Stock,sell,1001.0,Nancy Pelosi,TX-7,No,NaN,24.575148
100,2023,12/25/2022,2022-06-25,Spouse,AAPL,AAPL Stock,sell,100001.0,Nancy Pelosi,NY-14,Yes,NaN,33.088562


In [ ]:
# Calculating size based on amount and price
test['size'] = test['amount'] / test['price']
test

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol,price,size
0,2023,6/15/2023,2023-05-20,Spouse,AAPL,Apple Inc. Stock,buy,100001.0,Nancy Pelosi,CA-12,Yes,NaN,86.019421,1162.539798
1,2023,05-12-2023,2023-04-10,Self,GOOGL,Alphabet Inc. Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN,8.773509,5699.088209
2,2023,12/29/2023,2022-12-06,Dependent,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,NY-14,Yes,NaN,65.104681,1536.003233
3,2023,2/13/2022,2022-04-14,Self,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,FL-9,No,NaN,64.400713,1552.793361
4,2023,6/23/2023,2022-03-11,Joint,MSFT,MSFT Stock,buy,1001.0,Nancy Pelosi,CA-12,Yes,NaN,35.082036,28.533122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2023,3/15/2023,2023-07-26,Dependent,AAPL,AAPL Stock,sell,1001.0,Nancy Pelosi,FL-9,Yes,NaN,87.272859,11.469774
98,2022,06-08-2022,2023-05-03,Joint,TSLA,TSLA Stock,sell,50001.0,Nancy Pelosi,CA-12,No,NaN,33.837085,1477.698210
99,2023,7/24/2022,2023-09-16,Self,GOOGL,GOOGL Stock,sell,1001.0,Nancy Pelosi,TX-7,No,NaN,24.575148,40.732206
100,2023,12/25/2022,2022-06-25,Spouse,AAPL,AAPL Stock,sell,100001.0,Nancy Pelosi,NY-14,Yes,NaN,33.088562,3022.222568


In [ ]:
# Selecting required columns and rename them
test = test[['transactionDate', 'ticker', 'type', 'size', 'price']]
test.columns = ['Date', 'Symbol', 'Side', 'Size', 'Price']

In [ ]:
test

,Date,Symbol,Side,Size,Price
0,2023-05-20,AAPL,buy,1162.539798,86.019421
1,2023-04-10,GOOGL,sell,5699.088209,8.773509
2,2022-12-06,AMZN,buy,1536.003233,65.104681
3,2022-04-14,AMZN,buy,1552.793361,64.400713
4,2022-03-11,MSFT,buy,28.533122,35.082036
...,...,...,...,...,...
97,2023-07-26,AAPL,sell,11.469774,87.272859
98,2023-05-03,TSLA,sell,1477.698210,33.837085
99,2023-09-16,GOOGL,sell,40.732206,24.575148
100,2022-06-25,AAPL,sell,3022.222568,33.088562


In [ ]:
# Sort by date
test = test.sort_values('Date')
test

,Date,Symbol,Side,Size,Price
21,2022-01-08,MSFT,sell,2250.539338,6.665513
31,2022-02-09,AAPL,sell,19.638443,50.971455
48,2022-02-27,AAPL,sell,3236.891774,77.234896
87,2022-03-07,AAPL,sell,5185.866407,48.208145
8,2022-03-10,AMZN,buy,16.058989,62.332693
...,...,...,...,...,...
35,2023-12-11,MSFT,sell,1215.968421,41.120311
55,2023-12-11,GOOGL,sell,19.137350,52.306092
44,2023-12-12,AAPL,buy,4148.265991,60.266386
68,2023-12-18,GOOGL,buy,8476.407290,29.493746


In [ ]:
performance_metrics = trade_perf(test)

         Date Symbol  Side         Size      Price         Return
21 2022-01-08   MSFT  sell  2250.539338   6.665513 -177204.193369
31 2022-02-09   AAPL  sell    19.638443  50.971455      21.008383
48 2022-02-27   AAPL  sell  3236.891774  77.234896   34679.335424
87 2022-03-07   AAPL  sell  5185.866407  48.208145  154760.942267
8  2022-03-10   AMZN   buy    16.058989  62.332693     327.104877
..        ...    ...   ...          ...        ...            ...
35 2023-12-11   MSFT  sell  1215.968421  41.120311   42569.212886
55 2023-12-11  GOOGL  sell    19.137350  52.306092     654.399093
44 2023-12-12   AAPL   buy  4148.265991  60.266386   92342.075582
68 2023-12-18  GOOGL   buy  8476.407290  29.493746  579271.970691
19 2023-12-29   AMZN   buy    12.541341  79.816025    -464.024396

[102 rows x 6 columns]


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)


In [ ]:
for metric, value in performance_metrics.items():
    print(f"{metric}: {value}")

Nancy Pelosi's Trading Performance Metrics:
Sharpe: -0.16
max-drawdown: -inf
profit factor: 0.39
total net profit: -3723165.58
Percent Profitable: 47.06
average win\loss Ratio: -0.44
Expectancy: 83416.57
Recovery factor: -0.00
volatility: 232240.74
ROI: -45.42


In [ ]:
test.head()

,Date,Symbol,Side,Size,Price
21,2022-01-08,MSFT,sell,2250.539338,6.665513
31,2022-02-09,AAPL,sell,19.638443,50.971455
48,2022-02-27,AAPL,sell,3236.891774,77.234896
87,2022-03-07,AAPL,sell,5185.866407,48.208145
8,2022-03-10,AMZN,buy,16.058989,62.332693
